# 필요한 라이브러리 불러오기
* 실습을 위한 pytorch라이브러리를 불러옵니다.
* https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/GAN_for_MNIST_Tutorial.ipynb

In [1]:
import torch
import torch.nn as nn

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

/Users/isang-yun/PycharmProjects/AiPaper_samplecode/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# 생성자 및 판별자 모델 정의

In [2]:
latent_dim = 100

''' 생성자 (Generator) 클래스 정의 '''
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        '''하나의 블록(block)정의'''
        def block(input_dim, output_dim, normalize=True):
            layers = [nn.Linear(input_dim, output_dim)]
            if normalize:
                '''배치 정규화 (batch normalization) 수행 (차원 동일)'''
                layers.append(nn.BatchNorm1d(output_dim, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        '''생성자 모델은 연속 적인 여러 개의 블록을 가짐'''
        '''*연산자는 unpacking??'''
        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, 1 * 28 * 28),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img

In [3]:
'''판별자 (Discriminator) 클래스 정의'''
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(1 * 28 * 28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    '''이미지에 대한 판별 결과를 반환'''
    def forward(self, img):
        flattend = img.view(img.size(0), -1)
        output = self.model(flattend)
        return output

# 학습 데이터 셋 불러오기
* 학습을 위해 MINIST 데이터셋을 불러옵니다.

In [4]:
transform_train = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = datasets.MNIST(
    root="./dataset",
    train=True,
    download=True,
    transform=transform_train
)
dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=4
)

# 모델 학습 및 샘플링
* 학습을 위해 생성자와 판별자 모델을 초기화합니다.
* 적절한 하이퍼 파라미터를 설정합니다.

In [5]:
'''생성사 (generator)와 판별자(discriminatior) 초기화'''
generator = Generator()
discriminator = Discriminator()

generator.cpu()
discriminator.cpu()

'''손실 함수(loss function)'''
adversarial_loss = nn.BCELoss()
adversarial_loss.cpu()

'''학습률(learning rate) 설정'''
lr = 0.0002

'''생성자와 판별자를 위한 최적화 함수'''
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

In [8]:
import time

'''학습의 횟수(epoch) 설정'''
n_epochs = 200
'''몇 번째 배치(batch)마다 결과를 출력할 것인지 설정'''
sample_interval = 2000
start_time = time.time()

for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        '''진짜 (real)이미지와 가짜 (fake)이미지에 대한 정답 레이블 생성'''
        '''진짜 (real) : 1'''
        real = torch.FloatTensor(imgs.size(0), 1).fill_(1.0)
        '''가짜 (fake) : 0'''
        fake = torch.FloatTensor(imgs.size(0), 1).fill_(0.0)

        real_imgs = imgs

/Users/isang-yun/PycharmProjects/AiPaper_samplecode/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/isang-yun/PycharmProjects/AiPaper_samplecode/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/isang-yun/PycharmProjects/AiPaper_samplecode/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/isang-yun/PycharmProjects/AiPaper_samplecode/venv/lib/python3.9/site-packages/urllib3/_

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/isang-yun/PycharmProjects/AiPaper_samplecode/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/2n/m3c7rxxd2wq8j28g6l4d_9bw0000gn/T/ipykernel_26146/3437562017.py", line 10, in <module>
    for i, (imgs, _) in enumerate(dataloader):
  File "/Users/isang-yun/PycharmProjects/AiPaper_samplecode/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 633, in __next__
    data = self._next_data()
  File "/Users/isang-yun/PycharmProjects/AiPaper_samplecode/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in _next_data
    idx, data = self._get_data()
  File "/Users/isang-yun/PycharmProjects/AiPaper_samplecode/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1294, in _get_data
    success, data = self._try_get_data()
  File "/Users/isang-yun/PycharmProjects/AiPaper_sa